# Chat with PDF - test, evaluation and experimentation

We will walk you through how to use prompt flow Python SDK to test, evaluate and experiment with the "Chat with PDF" flow.

## 0. Install dependencies

In [ ]:
%pip install -r requirements.txt

## 1. Create connections
Connection in prompt flow is for managing settings of your application behaviors incl. how to talk to different services (Azure OpenAI for example).

In [ ]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

You will need to have a connection named "azure_open_ai_connection" to run the chat_with_pdf flow.

In [ ]:
# create needed connection
from promptflow.entities import AzureOpenAIConnection

try:
    conn_name = "azure_open_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<test_key>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

## 2. Test the flow

In [ ]:
output = pf.flows.test(
    ".",
    inputs={
        "chat_history": [],
        "pdf_url": "https://arxiv.org/pdf/1810.04805.pdf",
        "question": "what is BERT?",
    },
)
print(output)

## 3. Run the flow with a data file

In [ ]:
flow_path = "."
data_path = "./data/bert-paper-qna-3-line.jsonl"

config_2k_context = {
    "EMBEDDING_MODEL_DEPLOYMENT_NAME": "text-embedding-ada-002",
    "CHAT_MODEL_DEPLOYMENT_NAME": "gpt-35-turbo",
    "PROMPT_TOKEN_LIMIT": 2000,
    "MAX_COMPLETION_TOKENS": 256,
    "VERBOSE": True,
    "CHUNK_SIZE": 256,
    "CHUNK_OVERLAP": 32,
}

column_mapping = {
    "question": "${data.question}",
    "pdf_url": "${data.pdf_url}",
    "chat_history": "${data.chat_history}",
    "config": config_2k_context,
}
run_2k_context = pf.run(flow=flow_path, data=data_path, column_mapping=column_mapping)
pf.stream(run_2k_context)

print(run_2k_context)

In [44]:
pf.get_details(run_2k_context)

,inputs.chat_history,inputs.config,inputs.pdf_url,inputs.question,inputs.line_number,outputs.answer,outputs.context
0,[],{'EMBEDDING_MODEL_DEPLOYMENT_NAME': 'text-embe...,https://arxiv.org/pdf/1810.04805.pdf,What is the name of the new language represent...,0,The name of the new language representation mo...,"[Matthew Peters, Waleed Ammar, Chandra Bhagava..."
1,[],{'EMBEDDING_MODEL_DEPLOYMENT_NAME': 'text-embe...,https://arxiv.org/pdf/1810.04805.pdf,What is the main difference between BERT and p...,1,The main difference between BERT and previous ...,[BERT: Pre-training of Deep Bidirectional Tran...
2,[],{'EMBEDDING_MODEL_DEPLOYMENT_NAME': 'text-embe...,https://arxiv.org/pdf/1810.04805.pdf,What is the size of the vocabulary used by BERT?,2,The context does not provide information about...,[size of 32 and ﬁne-tune for 3 epochs over the...


# 4. Evaluate the "groundedness"
The [eval-groundedness flow](../../evaluation/eval-groundedness/) is using ChatGPT/GPT4 model to grade the answers generated by chat-with-pdf flow.

In [ ]:
eval_groundedness_flow_path = "../../evaluation/eval-groundedness/"
eval_groundedness_2k_context = pf.run(
    flow=eval_groundedness_flow_path,
    run=run_2k_context,
    column_mapping={
        "question": "${run.inputs.question}",
        "answer": "${run.outputs.answer}",
        "context": "${run.outputs.context}",
    },
    display_name="eval_groundedness_2k_context"
)
pf.stream(eval_groundedness_2k_context)

print(eval_groundedness_2k_context)

In [46]:
pf.get_details(eval_groundedness_2k_context)

,inputs.answer,inputs.context,inputs.question,inputs.line_number,outputs.groundedness
0,The name of the new language representation mo...,"[Matthew Peters, Waleed Ammar, Chandra Bhagava...",What is the name of the new language represent...,0,10
1,The main difference between BERT and previous ...,[BERT: Pre-training of Deep Bidirectional Tran...,What is the main difference between BERT and p...,1,10
2,The context does not provide information about...,[size of 32 and ﬁne-tune for 3 epochs over the...,What is the size of the vocabulary used by BERT?,2,1


In [47]:
pf.get_metrics(eval_groundedness_2k_context)

{'groundedness': 7.0}

In [48]:
pf.visualize(eval_groundedness_2k_context)

The HTML file is generated at '/private/var/folders/p1/msggn1bx4sl914zrsqqq5hw40000gn/T/pf-visualize-detail-r_r76wuy.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


You will see a web page like this. It gives you detail about how each row is graded and even the details how the evaluation run executes:
![pf-visualize-screenshot](./assets/pf-visualize-screenshot.png)

# 5. Try a different configuration and evaluate again - experimentation

NOTE: since we only use 3 lines of test data in this example, and because of the non-deterministic nature of LLMs, don't be surprised if you see exact same metrics when you run this process.

In [ ]:
config_3k_context = {
    "EMBEDDING_MODEL_DEPLOYMENT_NAME": "text-embedding-ada-002",
    "CHAT_MODEL_DEPLOYMENT_NAME": "gpt-35-turbo",
    "PROMPT_TOKEN_LIMIT": 3000,
    "MAX_COMPLETION_TOKENS": 256,
    "VERBOSE": True,
    "CHUNK_SIZE": 256,
    "CHUNK_OVERLAP": 32,
}

run_3k_context = pf.run(flow=flow_path, data=data_path, column_mapping=column_mapping)
pf.stream(run_3k_context)

print(run_3k_context)

In [ ]:
eval_groundedness_3k_context = pf.run(
    flow=eval_groundedness_flow_path,
    run=run_3k_context,
    column_mapping={
        "question": "${run.inputs.question}",
        "answer": "${run.outputs.answer}",
        "context": "${run.outputs.context}",
    },
    display_name="eval_groundedness_3k_context"
)
pf.stream(eval_groundedness_3k_context)

print(eval_groundedness_3k_context)

In [51]:
pf.get_details(eval_groundedness_3k_context)

,inputs.answer,inputs.context,inputs.question,inputs.line_number,outputs.groundedness
0,The name of the new language representation mo...,"[Matthew Peters, Waleed Ammar, Chandra Bhagava...",What is the name of the new language represent...,0,10
1,The main difference between BERT and previous ...,[BERT: Pre-training of Deep Bidirectional Tran...,What is the main difference between BERT and p...,1,10
2,"Based on the provided context, the size of BER...",[size of 32 and ﬁne-tune for 3 epochs over the...,What is the size of the vocabulary used by BERT?,2,10


In [52]:
pf.visualize([eval_groundedness_2k_context, eval_groundedness_3k_context])

The HTML file is generated at '/private/var/folders/p1/msggn1bx4sl914zrsqqq5hw40000gn/T/pf-visualize-detail-ru1nxoek.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
